<a href="https://colab.research.google.com/github/DeeeTeeee/machinelearning/blob/master/Credit_Card_Prediction_with_pycaret_82_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install pycaret
#!pip install pycaret[experimental]
#!pip install gradio

In [4]:
from pycaret.classification import *
import pandas as pd

In [5]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Load the dataset and display some values
train_df = pd.read_csv('/content/drive/MyDrive/emma/archive/train_20D8GL3.csv')
test_df = pd.read_csv('/content/drive/MyDrive/emma/archive/test_O6kKpvt.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/emma/archive/sample_submission_gm6gE0l.csv')

In [7]:
# Drop the ID column from both dataframes
train_df = train_df.drop("ID", axis=1)
test_df = test_df.drop("ID", axis=1)

## Data Understanding

In [8]:
train_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,360000,2,2,2,25,-1,-1,-1,-1,-1,...,3435,8870,2020,12590,3479,3446,8870,2020,27043,0
1,240000,2,1,2,46,2,2,-2,-1,0,...,2240,1681,2267,0,0,2240,0,2267,3074,0
2,320000,2,2,1,41,0,0,0,0,0,...,35409,35036,35939,5000,2000,2000,3000,3000,1500,0
3,50000,2,2,2,24,-1,-1,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1
4,80000,1,3,1,52,-1,0,0,0,0,...,20561,21261,21457,1500,1500,1500,1028,876,1449,0


In [9]:
test_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,60000,2,2,1,30,0,0,0,0,0,...,38117,38874,38667,39544,1628,1618,1375,1403,1520,1205
1,200000,2,1,2,29,0,0,0,0,2,...,76593,93988,50292,10510,4160,14593,20150,0,5739,12050
2,230000,1,1,2,39,0,0,0,0,-2,...,20650,0,0,0,2500,2000,0,0,0,0
3,150000,1,1,2,40,0,0,0,0,0,...,145978,139791,142745,146872,5800,5245,5200,5200,6500,7200
4,140000,1,2,2,27,0,0,0,0,0,...,76230,77954,78953,81848,10000,10000,3393,3260,5000,3000


In [10]:
print(train_df.shape, test_df.shape)

(21000, 24) (9000, 23)


In [11]:
train_df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default_payment_next_month'],
      dtype='object')

Thouugh we have the text dataset, I want to have two datasets to test the model, so I will be spliting the `train_df` datasets into two and use the other part of this first and then later will use the `test_df`

In [12]:
data=train_df.sample(frac=0.95, random_state=786)

In [13]:
data.shape

(19950, 24)

In [14]:
data_unseen = train_df.drop(data.index)

In [15]:
data_unseen.shape

(1050, 24)

Let's change the index and arrange it

In [16]:
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
10670,150000,1,1,1,35,-1,-1,-1,-1,-1,...,4869,9157,9624,6879,16701,4883,9199,9652,443,0
16274,50000,2,2,2,23,-1,0,0,2,0,...,11449,9914,9875,1502,2651,500,500,500,500,0
20801,150000,2,2,1,37,0,0,0,0,0,...,46890,47873,48833,4960,1765,1679,1739,1757,1795,0
4425,90000,2,2,1,36,-1,-1,-1,-1,-1,...,850,1000,1360,0,490,850,1000,1360,0,1
11957,180000,2,1,2,29,0,0,0,0,0,...,9531,10131,0,1745,1200,1531,3500,0,0,0


In [17]:
data.reset_index(inplace=True, drop = True)
data_unseen.reset_index(inplace = True, drop = True)

In [18]:
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,150000,1,1,1,35,-1,-1,-1,-1,-1,...,4869,9157,9624,6879,16701,4883,9199,9652,443,0
1,50000,2,2,2,23,-1,0,0,2,0,...,11449,9914,9875,1502,2651,500,500,500,500,0
2,150000,2,2,1,37,0,0,0,0,0,...,46890,47873,48833,4960,1765,1679,1739,1757,1795,0
3,90000,2,2,1,36,-1,-1,-1,-1,-1,...,850,1000,1360,0,490,850,1000,1360,0,1
4,180000,2,1,2,29,0,0,0,0,0,...,9531,10131,0,1745,1200,1531,3500,0,0,0


In [19]:
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data for Prediction: ' + str(data_unseen.shape))
print('Original Test data: ' + str(test_df.shape))

Data for Modeling: (19950, 24)
Unseen Data for Prediction: (1050, 24)
Original Test data: (9000, 23)


In [20]:
clf01 = setup(data=data, target='default_payment_next_month', session_id=123)

,Description,Value
0,Session id,123
1,Target,default_payment_next_month
2,Target type,Binary
3,Original data shape,"(19950, 24)"
4,Transformed data shape,"(19950, 24)"
5,Transformed train set shape,"(13965, 24)"
6,Transformed test set shape,"(5985, 24)"
7,Numeric features,23
8,Preprocess,True
9,Imputation type,simple


In [21]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8184,0.7654,0.3787,0.6573,0.4803,0.3799,0.4012,0.7770
gbc,Gradient Boosting Classifier,0.8184,0.7836,0.3668,0.6648,0.4724,0.3737,0.3980,0.2090
lightgbm,Light Gradient Boosting Machine,0.8180,0.7777,0.3684,0.6627,0.4732,0.3739,0.3977,0.3620
ada,Ada Boost Classifier,0.8146,0.7784,0.3248,0.6710,0.4374,0.3421,0.3746,0.3770
et,Extra Trees Classifier,0.8127,0.7573,0.3765,0.6310,0.4713,0.3663,0.3845,1.2550
xgboost,Extreme Gradient Boosting,0.8109,0.7579,0.3616,0.6291,0.4591,0.3545,0.3745,0.1900
lda,Linear Discriminant Analysis,0.8088,0.7190,0.2535,0.6902,0.3706,0.2852,0.3353,0.1120
ridge,Ridge Classifier,0.7971,0.0000,0.1413,0.7218,0.2358,0.1758,0.2555,0.1130
dummy,Dummy Classifier,0.7780,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1100
lr,Logistic Regression,0.7778,0.6569,0.0000,0.0000,0.0000,-0.0004,-0.0035,0.4690


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [22]:
print(best_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)


In [23]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [24]:
# Create a Decision Tree Classifier model
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8182,0.7781,0.3645,0.6647,0.4708,0.3721,0.3967
1,0.8354,0.7771,0.4452,0.7041,0.5455,0.4511,0.4688
2,0.7996,0.7538,0.3290,0.5862,0.4215,0.3117,0.3307
3,0.8218,0.7720,0.3935,0.6667,0.4949,0.3953,0.4156
4,0.8082,0.7430,0.3452,0.6221,0.4440,0.3394,0.3609
5,0.8188,0.7758,0.3839,0.6575,0.4847,0.3838,0.4043
6,0.8245,0.7649,0.4032,0.6757,0.5051,0.4065,0.4266
7,0.8223,0.7717,0.3935,0.6703,0.4959,0.3968,0.4176
8,0.8173,0.7562,0.3742,0.6554,0.4764,0.3756,0.3973


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Create a Gradient Boosting Classifier model
gbc = create_model('gbc')

## Hyperparameter Tuning

In [ ]:
tune_rf = tune_model(rf)
print(rf)

In [ ]:
tune_gbc = tune_model(gbc)
print(gbc)

In [ ]:
tune_gbc

Let us pass the `data_unseen` and the `test_df` on the model...`rf`

In [25]:
unseen_pred = predict_model(rf,data = data_unseen )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8200,0.7685,0.3194,0.6216,0.4220,0.3282,0.3538


We can compare the `default_payment_next_month` column to the	`prediction_label` column to see the accuracy of the model:

In [26]:
unseen_pred.sample(30)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month,prediction_label,prediction_score
43,80000,1,3,1,43,2,-1,-1,2,-1,...,390,1473,3133,0,1473,390,390,0,1,0.53
239,610000,1,1,2,35,0,0,0,0,0,...,271886,10000,11000,10000,11000,10000,10000,0,0,0.85
511,160000,1,1,2,53,2,2,-1,-1,-2,...,2191,0,4582,0,0,2191,4825,1,1,0.54
558,130000,2,2,2,23,-1,0,-1,-1,-1,...,2940,1000,10363,5177,19040,2940,1332,0,0,0.87
336,200000,2,1,2,34,-1,-1,-1,-1,-1,...,0,11381,0,1000,0,0,0,0,0,0.88
375,310000,1,1,2,30,-1,0,0,0,0,...,60592,207000,20000,60126,20000,25000,6000,0,0,0.95
1021,140000,2,1,2,26,-1,-1,-1,-1,-1,...,0,2136,6573,0,770,0,0,0,0,0.90
308,30000,2,1,2,24,0,0,2,2,-2,...,0,2000,0,0,0,0,0,0,0,0.66
979,200000,2,2,2,31,1,-2,-1,-1,-2,...,0,0,500,0,0,0,0,0,0,0.79
817,150000,2,1,2,52,0,0,0,0,0,...,23126,5000,13003,15000,11000,10000,3000,0,0,0.89


Let us check the metrics

In [27]:
from pycaret.utils.generic import check_metric

In [28]:
accuracy = check_metric(unseen_pred['default_payment_next_month'], unseen_pred['prediction_label'],metric='Accuracy')
print('The accuracy is:', accuracy * 100)

The accuracy is: 82.0


Now let us make prediction on our `test_df` and also check the Accuracy:

In [29]:
# Make predictions on the test dataset using the best model
test_df_pred = predict_model(rf, data=test_df)

In [30]:
test_df_pred

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,prediction_label,prediction_score
0,60000,2,2,1,30,0,0,0,0,0,...,38667,39544,1628,1618,1375,1403,1520,1205,0,0.68
1,200000,2,1,2,29,0,0,0,0,2,...,50292,10510,4160,14593,20150,0,5739,12050,0,0.85
2,230000,1,1,2,39,0,0,0,0,-2,...,0,0,2500,2000,0,0,0,0,0,0.88
3,150000,1,1,2,40,0,0,0,0,0,...,142745,146872,5800,5245,5200,5200,6500,7200,0,0.98
4,140000,1,2,2,27,0,0,0,0,0,...,78953,81848,10000,10000,3393,3260,5000,3000,0,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,230000,2,2,1,42,-1,-1,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0.81
8996,90000,2,1,2,28,-1,-1,-1,-1,-1,...,0,5990,1950,7956,499,0,5990,0,0,0.91
8997,60000,1,2,2,26,0,0,0,0,0,...,26958,28847,2282,2324,2049,2000,3000,1120,0,0.86
8998,70000,2,2,2,23,0,0,0,0,0,...,48212,45849,2100,2079,2500,2000,2000,1700,0,0.91


In [31]:
test_df_accuracy = check_metric(test_df_pred['prediction_label'], test_df_pred['prediction_score'],metric='Accuracy')
print('The accuracy is:', test_df_accuracy * 100)

The accuracy is: 87.16000000000001


In [32]:
import pickle

# Save the model
with open('rf.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [34]:
# train a model
rf = create_model('rf')

# create app
create_app(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8182,0.7781,0.3645,0.6647,0.4708,0.3721,0.3967
1,0.8354,0.7771,0.4452,0.7041,0.5455,0.4511,0.4688
2,0.7996,0.7538,0.3290,0.5862,0.4215,0.3117,0.3307
3,0.8218,0.7720,0.3935,0.6667,0.4949,0.3953,0.4156
4,0.8082,0.7430,0.3452,0.6221,0.4440,0.3394,0.3609
5,0.8188,0.7758,0.3839,0.6575,0.4847,0.3838,0.4043
6,0.8245,0.7649,0.4032,0.6757,0.5051,0.4065,0.4266
7,0.8223,0.7717,0.3935,0.6703,0.4959,0.3968,0.4176
8,0.8173,0.7562,0.3742,0.6554,0.4764,0.3756,0.3973


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>